In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fastai.tabular.all import *
from sklearn.model_selection import KFold, GroupKFold
from optuna.integration import FastAIPruningCallback
import optuna

In [3]:
STOCK_COUNT = 112



In [4]:
config = {
 'block_size': 566,
 'bottleneck': 89,
 'emb_p': 0.22892755376131763,
 'emb_size': 29,
 'jit_std': 0.044428121554388224,
 'lr': 0.0090050544233066,
 'mask_perc': 15,
 'multiplier0': 0.36610514833346075,
 'multiplier1': 0.4699461659624539,
 'multiplier2': 0.19267216205111673,
 'multiplier3': 0.1103899309385393,
 'p0': 0.7779388520646903,
 'p1': 0.4268044282083482,
 'p2': 0.017189932979278854,
 'p3': 0.6956976227194849,
 'time_p0': 0.040130990403726904,
 'time_p1': 0.2795015767348503,
 'time_p2': 0.2534561744168111,
 'time_p3': 0.35356055022302363,
 'wd': 0.1755624424764211
}

In [5]:
def fill_missing(train_df):
    all_times = train_df.time_id.unique()
    all_stocks = train_df.stock_id.unique()
    filled_df = train_df.copy()
    filled_df=filled_df.set_index(['time_id', 'stock_id'])
    new_index = pd.MultiIndex.from_product([all_times, all_stocks], names = ['time_id', 'stock_id'])
    filled_df = filled_df.reindex(new_index).reset_index()
    filled_df = filled_df.fillna(0)
    return filled_df


def tauify(train_df):
    for c in train_df.columns:
        if 'sum' in c: train_df[c] = np.sqrt(1/(train_df[c]+1))
    return train_df

def post_process(train_df, time_windows,  do_tau):
    train_df = fill_missing(train_df)
    
    if do_tau: train_df = tauify(train_df)
    return train_df

In [6]:
class Jitter(ItemTransform):
    def __init__(self, jit_std):
            super().__init__()
            self.split_idx = 0
            self.jit_std = jit_std
            
    def encodes(self, b):
        #print('doing jitter ', self.jit_std)
        jitter = torch.empty_like(b[1]).normal_(0, self.jit_std)
        b[1] += jitter
        return b

class MaskTfm(ItemTransform):
    
    def __init__(self, mask_perc):
        super().__init__()
        self.split_idx = 0
        self.mask_perc = mask_perc
    
    def mask(self, x, indices):
        x[torch.tensor(indices, device=x.device)] = 0
        return x
    
    def encodes(self, x):
        #print('doing mask', self.mask_perc)
        n = len(x[0])
        to_mask = (n * self.mask_perc) // 100
        indices = np.random.choice(np.array(range(n)), to_mask, replace=False)
        x = [self.mask(y, indices) for y in x]
        
        return x

class MyDataLoader(TabDataLoader):
    def __init__(self, dataset, jit_std, mask_perc, bs=16, shuffle=False, after_batch=None, num_workers=0,  **kwargs):
        if after_batch is None: after_batch = L(TransformBlock().batch_tfms)+ReadTabBatch(dataset) + [Jitter(jit_std), MaskTfm(mask_perc)]
        super().__init__(dataset, bs=bs, shuffle=shuffle, after_batch=after_batch, num_workers=num_workers, **kwargs)

    def shuffle_fn(self, idxs):
        idxs = np.array(idxs).reshape(-1,112)
        np.random.shuffle(idxs)
        return idxs.reshape(-1).tolist()

def get_dls(train_df, bs, trn_idx, val_idx, jit_std=.13, mask_perc=8):
    cont_nn,cat_nn = cont_cat_split(train_df, max_card=9000, dep_var='target')
    cat_nn=[x for x in cat_nn if not x in ['row_id', 'time_id']]
    
    procs_nn = [Categorify, Normalize]
    to_nn = TabularPandas(train_df, procs_nn, cat_nn, cont_nn, splits=[list(trn_idx), list(val_idx)], y_names='target')
    dls = to_nn.dataloaders(bs=112*100, shuffle=True, dl_type = MyDataLoader, jit_std=jit_std, mask_perc=mask_perc)
    dls.train_ds.split_idx=0
    dls.valid_ds.split_idx=1
    return dls

In [7]:
#train_df = pd.read_feather('train_24cols.feather')
# train_df = pd.read_feather('train_126ftrs.feater')
# train_df = fill_missing(train_df)
# train_df = append_trade_count(train_df)


In [8]:
class TimeEncoding(nn.Module):
    def __init__(self, inp_size, bottleneck, p):
        super().__init__()
        self.multiplier  = nn.Parameter(torch.tensor(.5)) 
        self.initial_layers = LinBnDrop(inp_size, bottleneck, act=nn.ReLU(True), p=p, bn=False)
        
        self.concat_layers = nn.Sequential(
            nn.BatchNorm1d(bottleneck * STOCK_COUNT),
            nn.Linear(bottleneck * STOCK_COUNT, inp_size),
            nn.Tanh()
        )
        
    def forward(self, x):
        y = self.initial_layers(x)
        times = y.shape[0] // STOCK_COUNT
        y = y.view(times, -1)
        y = self.concat_layers(y)
   
        y = y.view(times,1,-1).expand(times,STOCK_COUNT,-1).contiguous().view(times*STOCK_COUNT, -1)
        
        return x + y * self.multiplier

class BN(nn.Module):
    def __init__(self, features):
        super().__init__()
        self.num_features = features
        self.bn = nn.BatchNorm1d(STOCK_COUNT * self.num_features)
    def forward(self, x):
        sh = x.shape
        x = x.view(-1, STOCK_COUNT * self.num_features)
        x = self.bn(x)
        return x.view(*sh)

class ParallelBlock(nn.Module):
    def __init__(self, block_size, p, time_p, bottleneck, do_skip):
        super().__init__()
        self.do_skip = do_skip
        self.layers = nn.Sequential(
            nn.Linear(block_size, block_size),
            BN(block_size ),
            nn.Dropout(p),
            nn.ReLU(True),
            TimeEncoding(block_size, bottleneck, time_p)
        )
    def forward(self, x):
        y = self.layers(x)
        if self.do_skip: return (y + x) /2
        else: return y
    
class ParallelModel(nn.Module):
    def __init__(self, inp_size, emb_sz, block_size, ps, bottleneck, time_ps, embed_p, do_skip ):
        super().__init__()
        
        self.embeds = nn.Parameter(torch.empty(STOCK_COUNT, emb_sz))
        torch.nn.init.normal_(self.embeds)
        self.embed_drop = nn.Dropout(embed_p)
        
        layers = [nn.Linear(inp_size+emb_sz, block_size),
                 BN(block_size),
                 nn.ReLU(True)]
        for p, time_p in zip( ps, time_ps):            
            layers.append(ParallelBlock(block_size, p, time_p, bottleneck, do_skip))
            
        layers.append(nn.Linear(block_size, 1))
        layers.append(SigmoidRange(0, .1))
        self.layers = nn.Sequential(*layers)
    
    
    def forward(self, x_cat, x_cont):
        bs = x_cont.shape[0]//STOCK_COUNT
        emb = self.embeds.expand(bs, *self.embeds.shape)
    
        emb = self.embed_drop(emb)
        x_cont = x_cont.view(bs, STOCK_COUNT, -1)
        x = torch.cat([x_cont, emb], dim=2)
        x = x.view(bs*STOCK_COUNT, -1)
        return self.layers(x)

In [9]:
def rmspe(preds, targs):
    mask = targs != 0
    targs, preds = torch.masked_select(targs, mask), torch.masked_select(preds, mask)
    x = (targs-preds)/targs
    res= (x**2).mean().sqrt()
    if torch.isnan(res): 
        print(targs)
        print(preds)
        raise Exception('fck loss is nan')
    return res

In [10]:
def train( train_df, trn_idx=None, val_idx=None, save_as=None):
    
    jit_std=config['jit_std']
    mask_perc=config['mask_perc']
    
    
    if trn_idx is None:
        trn_idx, val_idx = first(GroupKFold().split(train_df, groups = train_df.time_id))
    dls = get_dls(train_df, 100, trn_idx, val_idx, jit_std=jit_std, mask_perc = mask_perc)
    inp_size = len(dls.cont_names)
    
    do_skip = True
    emb_size = config['emb_size']
   
    emb_p = config[f'emb_p']
    block_size = config['block_size']
    ps = [config[f'p{i}'] for i in range(4)]
    bottleneck = config['bottleneck']
    time_ps = [config[f'time_p{i}'] for i in range(4)]
    multipliers = [config[f'multiplier{i}'] for i in range(4)]
    
    lr = config['lr']
    wd = config['wd']
    
    
    model = ParallelModel(inp_size, emb_size, block_size, ps, bottleneck, time_ps, emb_p, do_skip)
    learn = Learner(dls,model = model, loss_func=rmspe, metrics=AccumMetric(rmspe), opt_func=ranger,wd=wd).to_fp16()
    
    learn.fit_flat_cos(50, lr)
    if save_as:
        learn.save(save_as)
    return  L(learn.recorder.values).itemgot(2)[-1]
    

def train_cross_valid(trial, train_df, save_as=None):
    res = 0
    do_subtract = False#trial.suggest_categorical('do_subtract', [True, False])
    do_append = False#trial.suggest_categorical('do_append', [True, False])
    do_tau = True#trial.suggest_categorical('do_tau', [True, False])
    train_df = post_process(train_df, time_windows, do_subtract, do_append, do_tau)
    splits = GroupKFold().split(train_df, groups = train_df.time_id)
    for idx, (trn_idx, val_idx) in enumerate(splits):
        v = train(trial, train_df, trn_idx, val_idx, save_as + str(idx) if save_as else None)
        print(f'fold {idx}: {v}')
        res +=v;
    return res/5

## Generate data

In [11]:
from optiver_features import *

In [12]:
%%time
# book_feature_dict = {
#     wap1: [np.mean, np.std, 'nunique'],
#     wap2: [np.mean, np.std],
#     log_return1: [np.std],
#     log_return2: [np.std],
#     ask_spread: [np.mean, np.std],
#     price_spread:[np.mean, np.std],
#     total_volume:[np.mean, np.std],
# }
# trade_feature_dict = {
#         log_return_price: [np.std, np.mean],
#         'seconds_in_bucket':[np.size],
#         'size':[np.sum],
#         'order_count':[np.sum],
# }

time_windows = [(0,600), (0,100), (100,200), (200,300), (300,400), (400, 500), (500,600)]
# ofg = OptiverFeatureGenerator(book_feature_dict, trade_feature_dict, time_windows)
# train_df = ofg.generate_train_df()






CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.58 µs


In [13]:
#train_df.to_feather('train_141cols.feather')

In [14]:
train_df = pd.read_feather('train_141cols.feather')

In [15]:
train_df = post_process(train_df, time_windows, True)

## Train

In [17]:
train(train_df)

epoch,train_loss,valid_loss,rmspe,time
0,3.582103,0.809872,0.810512,00:01
1,1.718419,0.566879,0.567700,00:02
2,1.117208,0.541401,0.541874,00:02
3,0.852414,0.504048,0.504526,00:02
4,0.714223,0.478481,0.478873,00:02
5,0.629962,0.409632,0.410347,00:02
6,0.568250,0.403058,0.403800,00:02
7,0.534728,0.437080,0.437478,00:02
8,0.530645,0.392528,0.393109,00:02
9,0.503139,0.407431,0.408438,00:01


0.21328474581241608